# Data Preprocessing for Visualization in Tableau

In [20]:
import json
import csv
import re

Meetup Data from Json to Csv

In [47]:
with open('meetup-20240407.json') as file:
    data = json.load(file)

csv_file = open('event.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)

count = 0
for item in data:
    if count == 0:
        header = item.keys()
        csv_writer.writerow(header)
        count += 1
    if '_id' in item and '$oid' in item['_id']:
        item['_id'] = item['_id']['$oid']
    csv_writer.writerow(item.values())

csv_file.close()

Posts from Json to Csv

In [5]:
with open('post-20240407.json') as file:
    data = json.load(file)

csv_file = open('post.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)

count = 0
for item in data:
    if count == 0:
        header = item.keys()
        csv_writer.writerow(header)
        count += 1
    csv_writer.writerow(item.values())

csv_file.close()

second post data 

In [7]:
with open('system_data.postd_ata.json') as file:
    data = json.load(file)

csv_file = open('1post.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)

count = 0
for item in data:
    if count == 0:
        header = item.keys()
        csv_writer.writerow(header)
        count += 1
    if '_id' in item and '$oid' in item['_id']:
        item['_id'] = item['_id']['$oid']
    csv_writer.writerow(item.values())

csv_file.close()

In [ ]:
user data

In [8]:
with open('system_data.user_info.json') as file:
    data = json.load(file)

csv_file = open('user-data.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)

count = 0
for item in data:
    if count == 0:
        header = item.keys()
        csv_writer.writerow(header)
        count += 1
    if '_id' in item and '$oid' in item['_id']:
        item['_id'] = item['_id']['$oid']
    csv_writer.writerow(item.values())

csv_file.close()

In [ ]:
gym data

In [23]:
import re
import html

In [49]:
def clean_json_string(raw_json):
    
    decoded_json = html.unescape(raw_json)
    decoded_json = re.sub(r'\\u003c[a-zA-Z\/][^>]*\\u003e', '', decoded_json)
    decoded_json = decoded_json.replace('\\"', '"')
    decoded_json = decoded_json.replace('\\n', '')
    decoded_json = decoded_json.replace('\\\\', '\\')
    decoded_json = re.sub(r'\\', '', decoded_json)
    decoded_json = re.sub(r'(?<=\})\s*(?=\{)', '},{', decoded_json)

    last_brace_pos = decoded_json.rfind('}')
    if last_brace_pos != -1:
        decoded_json = decoded_json[:last_brace_pos + 1]

    return '[' + decoded_json + ']'

def extract_json_data(filepath):
    try:
        with open(filepath, 'r') as file:
            content = file.read()

        json_start_pos = content.find('{')
        if json_start_pos == -1:
            print("No JSON object found in the file.")
            return None
        
        json_content = content[json_start_pos:]
        clean_content = clean_json_string(json_content)
        return json.loads(clean_content)
    except json.JSONDecodeError as e:
        print(f"JSON Decode Error: {e}")
        error_position = e.pos
        print("Error context (100 characters around error):", clean_content[max(0, error_position-50):error_position+50])
        return None
    except Exception as e:
        print(f"Error reading or parsing file: {e}")
        return None

def convert_json_to_csv(json_data, output_csv_path):
    try:
        df = pd.json_normalize(json_data)
        df.to_csv(output_csv_path, index=False)
        print(f"Data successfully saved to {output_csv_path}")
    except Exception as e:
        print(f"Error converting JSON to CSV: {e}")

json_data = extract_json_data('gym-data.txt')
if json_data:
    convert_json_to_csv(json_data, 'output.csv')

Data successfully saved to output.csv


In [ ]:
museum data

In [41]:
def convert_json_to_csv(input_file_path, output_file_path):
    # Open the input file and read the data
    with open(input_file_path, 'r') as file:
        data = file.read()

    # Try to find the JSON object starting and ending points
    start_index = data.find('{')
    end_index = data.rfind('}') + 1  # Include the last curly brace
    
    # Extract the JSON substring
    json_data = data[start_index:end_index]

    # Attempt to parse the JSON data
    try:
        parsed_json = json.loads(json_data)
        results = parsed_json.get('results', [])
    except json.JSONDecodeError as e:
        print("Failed to decode JSON:", e)
        return

    # Define the fields to be included in the CSV
    fields = [
        'name', 'business_status', 'formatted_address', 'latitude', 'longitude',
        'rating', 'user_ratings_total'
    ]

    # Write to the CSV file
    with open(output_file_path, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fields)
        writer.writeheader()
        for item in results:
            row = {
                'name': item['name'],
                'business_status': item['business_status'],
                'formatted_address': item['formatted_address'],
                'latitude': item['geometry']['location']['lat'],
                'longitude': item['geometry']['location']['lng'],
                'rating': item.get('rating', 'N/A'),
                'user_ratings_total': item.get('user_ratings_total', 'N/A')
            }
            writer.writerow(row)

# Example usage
input_file_path = 'museum-data.txt'
output_file_path = 'museum_data.csv'
convert_json_to_csv(input_file_path, output_file_path)

Failed to decode JSON: Extra data: line 61 column 8 (char 2053)


## Generating date data for user data

In [55]:
import numpy as np

In [56]:
file_path = 'tableau-data/user-data.csv'
data = pd.read_csv(file_path)

num_rows = data.shape[0]
chunks = np.random.randint(10, 16, size=(num_rows // 10) + 1)
date_range = pd.date_range(start='2023-01-01', periods=len(chunks), freq='D')

expanded_dates = np.repeat(date_range, chunks)[:num_rows]
formatted_dates = pd.to_datetime(expanded_dates).strftime('%d/%m/%Y')

data['date'] = formatted_dates

output_file_path = 'tableau-data/user-data.csv'
data.to_csv(output_file_path, index=False)

In [58]:
df = pd.read_csv('tableau-data/user-data.csv')
df['date'] = np.random.permutation(df['date'])
df.to_csv('tableau-data/user-data.csv', index=False)

In [60]:
file_path ='tableau-data/user-data.csv'
data = pd.read_csv(file_path)

num_rows = data.shape[0]

weights = [0.02]*6 + [0.03]*6 + [0.04]*6 + [0.15]*6
weights_normalized = [weight/sum(weights) for weight in weights]

hours = np.random.choice(a=[*range(24)], size=num_rows, p=weights_normalized)
minutes = np.random.randint(0, 60, size=num_rows)
evening_timestamps = ["{:02d}:{:02d}".format(h, m) for h, m in zip(hours, minutes)]

data['evening_timestamp'] = evening_timestamps
data.to_csv('tableau-data/user-data.csv', index=False)